In [1]:
import asyncio
import datetime as dt
import math
from typing import Literal

import matplotlib.pyplot as plt
import mplfinance as mpf
import numpy as np
import pandas as pd
import pandas_market_calendars as mcal
import plotly.graph_objects as go
import polars as pl
from dash import Dash, dcc, html
from plotly.subplots import make_subplots

nse = mcal.get_calendar("NSE")

In [2]:
pd.set_option("display.max_rows", 25_000)
pd.set_option("display.max_columns", 500)
pl.Config.set_tbl_cols(500)
pl.Config.set_tbl_rows(10_000)

pd.options.display.float_format = "{:.4f}".format

In [3]:
import sys

sys.path.append("..")
from tooling.enums import AssetClass, Index, Spot, StrikeSpread
from tooling.fetch import fetch_option_data, fetch_spot_data
from tooling.filter import find_atm, option_tool

In [4]:
async def get_expiry(f_today, index):

    if index == 'bnf':    
        if (f_today <= dt.date(2024, 1, 25)) and (f_today >= dt.date(2024, 1, 18)):
            f_expiry = dt.date(2024, 1, 25)
        elif (f_today <= dt.date(2024, 1, 31)) and (f_today >= dt.date(2024, 1, 26)):
            f_expiry = dt.date(2024, 1, 31)
        elif (f_today <= dt.date(2024, 2, 22)) and (f_today >= dt.date(2024, 2, 29)):
            f_expiry = dt.date(2024, 2, 29)
        elif (f_today <= dt.date(2024, 3, 25)) and (f_today >= dt.date(2024, 3, 27)):
            f_expiry = dt.date(2024, 2, 27)
        elif f_today < dt.date(2023, 9, 1):
            days_to_thursday = (3 - f_today.weekday()) % 7
            nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
            f_expiry = nearest_thursday
            if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
                f_expiry = nearest_thursday - dt.timedelta(days=1)
        elif f_today >= dt.date(2023, 9, 1):
            if f_today.day < 24:
                days_to_wednesday = (2 - f_today.weekday()) % 7
                nearest_wednesday = f_today + dt.timedelta(days=days_to_wednesday)
                f_expiry = nearest_wednesday
                if nse.valid_days(
                    start_date=nearest_wednesday, end_date=nearest_wednesday
                ).empty:
                    f_expiry = nearest_wednesday - dt.timedelta(days=1)
            else:
                days_to_thursday = (3 - f_today.weekday()) % 7
                nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
                f_expiry = nearest_thursday
                if nse.valid_days(
                    start_date=nearest_thursday, end_date=nearest_thursday
                ).empty:
                    f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'nifty':
        days_to_thursday = (3 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'finnifty' or index == 'fnf':
        days_to_thursday = (1 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

    elif index == 'midcpnifty' or index == 'midcp':
        days_to_thursday = (0 - f_today.weekday()) % 7
        nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
        f_expiry = nearest_thursday
        if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
            f_expiry = nearest_thursday - dt.timedelta(days=1)
        return f_expiry

async def get_expiry_nifty(f_today):

    days_to_thursday = (3 - f_today.weekday()) % 7
    nearest_thursday = f_today + dt.timedelta(days=days_to_thursday)
    f_expiry = nearest_thursday
    if nse.valid_days(start_date=nearest_thursday, end_date=nearest_thursday).empty:
        f_expiry = nearest_thursday - dt.timedelta(days=1)
    return f_expiry


async def get_option_contract_name(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"


def get_option_contract_name2(symbol, strike, expiry, opt_type):
    temp = "0"
    mth = expiry.month

    if (expiry + dt.timedelta(days=7)).month != expiry.month:
        date_string = expiry.strftime("%y%b").upper()
        return f"{symbol}{date_string}{strike}{opt_type}"
    else:
        if expiry.day <= 9:
            date_string = f"{expiry.year - 2000}{mth}{temp}{expiry.day}"
        else:
            date_string = f"{expiry.year - 2000}{mth}{expiry.day}"
        return f"{symbol}{date_string}{strike}{opt_type}"

In [5]:
bnf_1min = pd.read_csv("../data/nifty_wave.csv")
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["time"])
bnf_1min = bnf_1min[bnf_1min["datetime"].dt.year >= 2017]

In [6]:
bnf_1min.head()

,time,open,high,low,close,MA,Plot,Zero line,MACD Signal,datetime
3497,2017-01-02T09:15:00+05:30,8212.0000,8212.0000,8141.0000,8154.8000,8037.0038,51.3767,0,51.3767,2017-01-02 09:15:00+05:30
3498,2017-01-02T10:15:00+05:30,8155.1500,8161.8000,8133.8000,8146.8000,8040.7650,41.7773,0,41.7773,2017-01-02 10:15:00+05:30
3499,2017-01-02T11:15:00+05:30,8146.9500,8159.0500,8137.5000,8150.3000,8044.7463,35.4942,0,35.4942,2017-01-02 11:15:00+05:30
3500,2017-01-02T12:15:00+05:30,8150.8500,8166.5500,8144.9500,8165.3500,8049.3050,34.1609,0,34.1609,2017-01-02 12:15:00+05:30
3501,2017-01-02T13:15:00+05:30,8165.8000,8195.6000,8160.8000,8192.4500,8054.3700,38.7247,0,38.7247,2017-01-02 13:15:00+05:30


In [7]:
def resample(
    data: pl.DataFrame, timeframe, offset: dt.timedelta | None = None
) -> pl.DataFrame:
    return (
        data.set_sorted("datetime")
        .group_by_dynamic(
            index_column="datetime",
            every=timeframe,
            period=timeframe,
            label="left",
            offset=offset,
        )
        .agg(
            [
                pl.col("open").first().alias("open"),
                pl.col("high").max().alias("high"),
                pl.col("low").min().alias("low"),
                pl.col("close").last().alias("close"),
                pl.col("volume").sum().alias("volume"),
            ]
        )
    )


# ohlc_resampled = resample(pl.DataFrame(bnf_1min), '7d', pd.Timedelta(days=4))
# ohlc_resampled

In [8]:
bnf_1min["datetime"] = pd.to_datetime(bnf_1min["datetime"])
list_of_traded_dates = set(bnf_1min["datetime"].dt.date)
list_of_traded_dates

{datetime.date(2020, 1, 15),
 datetime.date(2022, 5, 17),
 datetime.date(2022, 11, 29),
 datetime.date(2017, 11, 24),
 datetime.date(2019, 8, 20),
 datetime.date(2017, 12, 14),
 datetime.date(2024, 9, 18),
 datetime.date(2023, 9, 20),
 datetime.date(2021, 10, 1),
 datetime.date(2024, 6, 26),
 datetime.date(2024, 10, 29),
 datetime.date(2019, 1, 8),
 datetime.date(2023, 11, 17),
 datetime.date(2024, 4, 30),
 datetime.date(2024, 3, 13),
 datetime.date(2019, 4, 30),
 datetime.date(2021, 8, 23),
 datetime.date(2023, 5, 5),
 datetime.date(2018, 5, 7),
 datetime.date(2017, 9, 19),
 datetime.date(2023, 5, 18),
 datetime.date(2024, 7, 1),
 datetime.date(2017, 1, 24),
 datetime.date(2020, 9, 21),
 datetime.date(2023, 9, 29),
 datetime.date(2018, 4, 25),
 datetime.date(2020, 10, 13),
 datetime.date(2022, 2, 21),
 datetime.date(2024, 1, 19),
 datetime.date(2020, 6, 25),
 datetime.date(2021, 5, 21),
 datetime.date(2022, 8, 29),
 datetime.date(2018, 6, 19),
 datetime.date(2022, 1, 3),
 datetime.dat

In [9]:
def rename_ohlc_columns(df: pl.DataFrame) -> pl.DataFrame:

    column_mapping = {"o": "open", "h": "high", "l": "low", "c": "close", "v": "volume"}
    df = df.rename(column_mapping)

    return df

In [10]:
PORTFOLIO_VALUE = 10_00_000 # 10 Lacs
# RPT_PCT = 0.01 # 1% RPT
SLIPPAGE_ = 0.0002
LEVERAGE_ = 5

In [11]:
def calculate_signals(df, ema_length, x_days, pct):
    # Calculate X-day High/Low and 20 EMA
    df['X_High'] = df['high'].rolling(x_days).mean()
    df['X_Low'] = df['low'].rolling(x_days).mean()
    df['EMA_20'] = df['close'].ewm(span=ema_length, adjust=False).mean()
    
    # Trend conditions
    df['EMA_Trend'] = df['EMA_20'] > df['EMA_20'].shift(1)
    
    # Signal conditions
    df['Buy_Signal'] = (
        (df['close'] > df['X_High'].shift(1) * (1 + (pct / 100))) & 
        (df['EMA_Trend']) & 
        (df['close'] > df['EMA_20'])
    )
    
    return df

In [12]:
# def backtest(df):
#     position = 0
#     entry_price = 0
#     trades = []
#     trailing_stop = None
    
#     for i in range(len(df)):
#         if df.loc[i, 'Buy_Signal'] and position == 0:
#             position = 1
#             # entry_price = df.loc[i, 'close']
#             trailing_stop_long = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Buy', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_long = df.loc[i, 'datetime']
#             entry_price_long = df.loc[i, 'close']
#             initial_sl_long = trailing_stop_long
        
#         elif df.loc[i, 'Sell_Signal'] and position == 0:
#             position = -1
#             entry_price = df.loc[i, 'close']
#             trailing_stop_short = df.loc[i, 'EMA_20']
#             # trades.append({'Type': 'Sell', 'Price': entry_price, 'Date': df.loc[i, 'datetime']})
#             entry_time_short = df.loc[i, 'datetime']
#             entry_price_short = df.loc[i, 'close']
#             initial_sl_short = trailing_stop_short
        
#         elif position == 1:
#             if df.loc[i, 'low'] < trailing_stop_long:
#                 # trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#                 exit_time_long = df.loc[i, 'datetime']
#                 exit_price_long = trailing_stop_long
#                 points_long = exit_price_long - entry_price_long
                
#             elif df.loc[i, 'Add_Long']:
#                 # trades.append({'Type': 'Add Long', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 entry_time_long = df.loc[i, 'datetime']
#                 entry_price_long = df.loc[i, 'close']
#                 initial_sl_long = trailing_stop_long
#                 trailing_stop_long = max(trailing_stop_long, df.loc[i, 'EMA_20'])
        
#         elif position == -1:
#             if df.loc[i, 'high'] > trailing_stop:
#                 trades.append({'Type': 'Exit', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 position = 0
#             elif df.loc[i, 'Add_Short']:
#                 trades.append({'Type': 'Add Short', 'Price': df.loc[i, 'close'], 'Date': df.loc[i, 'datetime']})
#                 trailing_stop = min(trailing_stop, df.loc[i, 'EMA_20'])
    
#     return pd.DataFrame(trades)

In [13]:
def backtest(df):
    # print(df.to_string())
    # return
    df.reset_index(drop=True, inplace=True)
    # Variables for long trades
    long_position = 0  # 0 = no position, 1 = long
    long_entry_price = 0
    long_entry_date = None
    long_trades = []
    tradebook = pd.DataFrame()
    tradebook_long = pd.DataFrame()
    # tradebook_short = pd.DataFrame()
    long_trailing_stop = None
    # Variables for short trades
    # short_position = 0  # 0 = no position, -1 = short
    # short_entry_price = 0
    # short_entry_date = None
    # short_trades = []
    # short_trailing_stop = None

    can_add_long = False
    # can_add_short = False

    trade_number = 0
    lock_initial_sl = False
    first_sl = 0

    lock_initial_sl_2 = False
    first_sl_2 = 0

    pyramid_number = 0
    can_pyramid = True
    pyramid_high = 0

    for i in range(1, len(df)):
        # Entry signals for long trades
        long_trailing_stop = df.loc[i, 'EMA_20']
        # short_trailing_stop = df.loc[i, 'EMA_20']
        
        if df.loc[i, 'Buy_Signal'] and long_position == 0:
            long_position = 1
            long_entry_price = df.loc[i, 'close']
            long_entry_date = df.loc[i, 'datetime']
            long_trailing_stop = df.loc[i, 'EMA_20']
            long_initial_sl = df.loc[i, 'X_Low']

            tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
                'Trade No.': trade_number,
                'Entry_Date': df.loc[i, 'datetime'],
                'Exit_Date': None,
                'Trade_Type': 'Long',
                'Entry_Price': df.loc[i, 'close'],
                'Previous MA Value': df.loc[i-1, 'X_High'],
                'Initial SL': long_initial_sl,
                'Exit_Price': None,
                # 'Profit/Loss': None  # Filled when exited
            }])], ignore_index=True)

            if not lock_initial_sl:
                first_sl = long_initial_sl
                lock_initial_sl = True

            can_pyramid = True
            pyramid_number = 0
            pyramid_high = 0
            continue
                
        # Exit or add to long position
        if long_position == 1:

            
            # y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
            # y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
            # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
            if df.loc[i, 'low'] <= long_initial_sl:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , long_initial_sl , 'Initial SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0
            
            elif df.loc[i, 'close'] <= long_trailing_stop:
                tradebook_long.loc[
                    (tradebook_long['Trade No.'] == trade_number),
                    ['Exit_Date', 'Exit_Price', 'Exit Remark']
                ] = [df.loc[i, 'datetime'] , df.loc[i, 'close'] , 'Trailing SL Hit']
                long_position = 0
                can_add_long = False
                trade_number += 1
                lock_initial_sl = False
                first_sl = 0
                can_pyramid = False
                pyramid_number = 0
                pyramid_high = 0

            # elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
            #     # Previous Y Candle Low Breached, confirmation found. Add position on high break
            #     can_add_long = True

            # if can_add_long and long_position and can_pyramid:
            #     if df.loc[i, 'high'] >= y_days_high and pyramid_number < pyr_num:
            #         tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
            #             'Trade No.': trade_number,
            #             'Entry_Date': df.loc[i, 'datetime'],
            #             'Exit_Date': None,
            #             'Trade_Type': 'Add_Long',
            #             'Entry_Price': y_days_high,
            #             'Initial SL': first_sl,
            #             'Exit_Price': None,
            #             # 'Profit/Loss': None  # Filled when exited
            #         }])], ignore_index=True)
            #         can_add_long = False
            #         pyramid_number += 1
            #         can_pyramid = True
    
    tradebook = pd.concat([tradebook_long, tradebook], ignore_index=True)
    return tradebook


In [14]:
# def backtest(df, y_days):
#     # Variables for long trades
#     long_position = 0  # 0 = no position, 1 = long
#     long_entry_price = 0
#     long_entry_date = None
#     long_trades = []
#     tradebook = pd.DataFrame()
#     tradebook_long = pd.DataFrame()
#     tradebook_short = pd.DataFrame()
#     long_trailing_stop = None
#     # Variables for short trades
#     short_position = 0  # 0 = no position, -1 = short
#     short_entry_price = 0
#     short_entry_date = None
#     short_trades = []
#     short_trailing_stop = None

#     can_add_long = False
#     can_add_short = False

#     trade_number = 0

#     for i in range(len(df)):
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']
        
#         if df.loc[i, 'Buy_Signal'] and long_position == 0:
#             long_position = 1
#             long_entry_price = df.loc[i, 'close']
#             long_entry_date = df.loc[i, 'datetime']
#             long_trailing_stop = df.loc[i, 'EMA_20']
#             long_initial_sl = df.loc[i, 'X_Low']

#             tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Long',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': long_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         # Exit or add to long position
#         if long_position == 1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'low'] <= long_initial_sl:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , long_initial_sl]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] <= long_trailing_stop:
#                 tradebook_long.loc[
#                     (tradebook_long['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 long_position = 0
#                 can_add_long = False
#                 trade_number += 1

#             elif (df.loc[i, 'low'] <= y_days_low) and not can_add_long:
#                 # Previous Y Candle Low Breached, confirmation found. Add position on high break
#                 can_add_long = True

#             if can_add_long and long_position:
#                 if df.loc[i, 'high'] >= y_days_high:
#                     tradebook_long = pd.concat([tradebook_long, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Long',
#                         'Entry_Price': y_days_high,
#                         'Initial SL': long_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_long = False

#     for i in range(len(df)):
        
#         # Entry signals for long trades
#         long_trailing_stop = df.loc[i, 'EMA_20']
#         short_trailing_stop = df.loc[i, 'EMA_20']

#         # Entry signals for short trades
#         if df.loc[i, 'Sell_Signal'] and short_position == 0:
#             short_position = -1
#             short_entry_price = df.loc[i, 'close']
#             short_entry_date = df.loc[i, 'datetime']
#             short_trailing_stop = df.loc[i, 'EMA_20']
#             short_initial_sl = df.loc[i, 'X_High']

#             tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                 'Trade No.': trade_number,
#                 'Entry_Date': df.loc[i, 'datetime'],
#                 'Exit_Date': None,
#                 'Trade_Type': 'Short',
#                 'Entry_Price': df.loc[i, 'close'],
#                 'Initial SL': short_initial_sl,
#                 'Exit_Price': None,
#                 # 'Profit/Loss': None  # Filled when exited
#             }])], ignore_index=True)

#         if short_position == -1:
            
#             y_days_low = df['low'].rolling(y_days).min().iloc[i-1]
#             y_days_high = df['high'].rolling(y_days).max().iloc[i-1]
            
#             # print(df.iloc[i]['datetime'], y_days_high, y_days_low)
            
#             if df.loc[i, 'high'] >= short_initial_sl:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] ,short_initial_sl]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1
            
#             elif df.loc[i, 'close'] >= short_trailing_stop:
#                 tradebook_short.loc[
#                     (tradebook_short['Trade No.'] == trade_number),
#                     ['Exit_Date', 'Exit_Price']
#                 ] = [df.loc[i, 'datetime'] , df.loc[i, 'close']]
#                 short_position = 0
#                 can_add_short = False
#                 trade_number += 1

#             elif (df.loc[i, 'high'] >= y_days_high) and not can_add_short:
#                 # Previous Y Candle High Breached, confirmation found. Add position on low break
#                 can_add_short = True

#             if can_add_short and short_position:
#                 if df.loc[i, 'low'] <= y_days_low:
#                     tradebook_short = pd.concat([tradebook_short, pd.DataFrame([{
#                         'Trade No.': trade_number,
#                         'Entry_Date': df.loc[i, 'datetime'],
#                         'Exit_Date': None,
#                         'Trade_Type': 'Add_Short',
#                         'Entry_Price': y_days_low,
#                         'Initial SL': short_initial_sl,
#                         'Exit_Price': None,
#                         # 'Profit/Loss': None  # Filled when exited
#                     }])], ignore_index=True)
#                     can_add_short = False
    
#     tradebook = pd.concat([tradebook_long, tradebook_short], ignore_index=True)
#     return tradebook


In [30]:
ema_length = 20
x_days = 3
# y_days = 4
pct = 0.25
RPT = 3
df = bnf_1min
df = calculate_signals(df, ema_length,x_days, pct)
# print(df.tail().to_string())
# df1 = df[df['Buy_Signal']]
# print(df1.to_string())
tb = backtest(df)
tb = tb.sort_values(by='Entry_Date')
# variation = f'EMA: {ema_length}, X: {x_days}, Y: {y_days}, PCT: {pct}%, RPT: {RPT}%'

/tmp/ipykernel_32292/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


In [31]:
import numpy as np
portfolio = 10000000
tb['PnL'] = np.where(
    tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
    tb['Exit_Price'] - tb['Entry_Price'],
    np.where(
        tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
        tb['Entry_Price'] - tb['Exit_Price'],
        0  # Default case if Trade_Type is something unexpected
    )
)

tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
tb['Qty'] = abs(RPT / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
# tb['Qty'] = portfolio * 8 / tb['Entry_Price']
tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
tb['Trade Year'] = tb['Entry_Date'].dt.year

In [32]:
# tb['Cumulative_PnL'] = tb['PnL w cs'].cumsum()

In [33]:
# tb.to_csv('GOLD TRADE CHECK JJPP.csv', index=False)

In [34]:
tb = tb.reset_index()
# tb

In [35]:
import matplotlib.pyplot as plt

def plot_cumulative_pnl(tb, title="Cumulative PnL Over Time"):
    if 'Cumulative_PnL' not in tb.columns:
        raise ValueError("The DataFrame must contain a 'Cumulative_PnL' column.")

    # Plot cumulative PnL
    plt.figure(figsize=(12, 3))
    plt.plot(tb.index, tb['Cumulative_PnL'], label='Cumulative PnL', color='blue', linewidth=2)
    plt.title(title, fontsize=8)
    plt.xlabel('Trade Index', fontsize=6)
    plt.ylabel('Cumulative PnL', fontsize=6)
    plt.legend(fontsize=6)
    plt.grid(alpha=0.5)
    plt.show()


In [36]:
# tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
# tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()

In [37]:
# tb_long_only = tb_long_only[tb_long_only['Trade Year'] >= 2017]

In [38]:
# plot_cumulative_pnl(tb_long_only)
# print(tb_long_only['PnL w cs'].sum())

In [39]:
# plot_cumulative_pnl(tb_short_only)
# print(tb_short_only['PnL w cs'].sum())

In [40]:
# plot_cumulative_pnl(tb)
# print(tb['PnL w cs'].sum())

In [41]:
# plot_cumulative_pnl(tb2)
# print(tb2['PnL w cs'].sum())

In [42]:
def generate_stats(tb_expiry, variation):
    stats_df8 = pd.DataFrame(
        index=range(2017, 2025),
        columns=[
            "Total ROI",
            "Total Trades",
            "Win Rate",
            "Avg Profit% per Trade",
            "Avg Loss% per Trade",
            "Max Drawdown",
            "ROI/DD Ratio",
            "Variation",
        ],
    )
    combined_df_sorted = tb_expiry
    # combined_df_sorted = tb_expiry_ce
    # combined_df_sorted = tb_expiry_pe
    
    # Iterate over each year
    for year in range(2017, 2025):
        # Filter trades for the current year
        year_trades = combined_df_sorted[(combined_df_sorted["Trade Year"] == year)]
    
        # Calculate total ROI
        total_roi = year_trades["ROI%"].sum()
    
        # Calculate total number of trades
        total_trades = len(year_trades)
    
        # Calculate win rate
        win_rate = (year_trades["ROI%"] > 0).mean() * 100
    
        # Calculate average profit per trade
        avg_profit = year_trades[year_trades["ROI%"] > 0]["ROI%"].mean()
    
        # Calculate average loss per trade
        avg_loss = year_trades[year_trades["ROI%"] < 0]["ROI%"].mean()
    
        # Calculate maximum drawdown
        max_drawdown = (
            year_trades["ROI%"].cumsum() - year_trades["ROI%"].cumsum().cummax()
        ).min()
    
        # Calculate ROI/DD ratio
        roi_dd_ratio = total_roi / abs(max_drawdown)

        variation = variation
    
        # Store the statistics in the DataFrame
        stats_df8.loc[year] = [
            total_roi,
            total_trades,
            win_rate,
            avg_profit,
            avg_loss,
            max_drawdown,
            roi_dd_ratio,
            variation,
        ]
    
    # Calculate overall statistics
    overall_total_roi = stats_df8["Total ROI"].sum()
    overall_total_trades = stats_df8["Total Trades"].sum()
    overall_win_rate = (combined_df_sorted["ROI%"] > 0).mean() * 100
    overall_avg_profit = combined_df_sorted[combined_df_sorted["ROI%"] > 0]["ROI%"].mean()
    overall_avg_loss = combined_df_sorted[combined_df_sorted["ROI%"] < 0]["ROI%"].mean()
    overall_max_drawdown = (
        combined_df_sorted["ROI%"].cumsum() - combined_df_sorted["ROI%"].cumsum().cummax()
    ).min()
    overall_roi_dd_ratio = overall_total_roi / abs(overall_max_drawdown)
    overall_variation = variation

    
    # Store the overall statistics in the DataFrame
    stats_df8.loc["Overall"] = [
        overall_total_roi,
        overall_total_trades,
        overall_win_rate,
        overall_avg_profit,
        overall_avg_loss,
        overall_max_drawdown,
        overall_roi_dd_ratio,
        overall_variation,
    ]
    
    # print(f'{overall_total_roi} , {overall_max_drawdown} , {overall_roi_dd_ratio}')
    
    return {overall_roi_dd_ratio: stats_df8}

In [43]:
tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
tb['Trade Year'] = tb['Entry_Date'].dt.year
# tb['ROI% w cs'] = tb['ROI%']

In [44]:
stats = generate_stats(tb, '...')
lol = pd.DataFrame()
for x, y in stats.items():
    lol = pd.DataFrame(y)

lol

,Total ROI,Total Trades,Win Rate,Avg Profit% per Trade,Avg Loss% per Trade,Max Drawdown,ROI/DD Ratio,Variation
2017,41.9279,57,35.0877,5.9858,-2.1024,-14.2530,2.9417,...
2018,58.9088,46,45.6522,5.3542,-2.1412,-16.0640,3.6671,...
2019,42.9348,55,32.7273,6.4110,-1.9584,-25.5116,1.6830,...
2020,111.5422,60,40.0000,8.0413,-2.2624,-14.3503,7.7728,...
2021,71.0830,62,37.0968,6.1513,-1.8051,-24.9277,2.8516,...
2022,44.1738,55,30.9091,7.1224,-2.0239,-23.8881,1.8492,...
2023,100.8319,48,43.7500,7.5237,-2.1172,-12.6956,7.9423,...
2024,49.8675,48,37.5000,5.8425,-1.9068,-16.0097,3.1148,...
Overall,521.2699,431,37.5870,6.5819,-2.0336,-25.5116,20.4326,...


# Gold 4H X_Days High, Y_Days Low 

In [45]:
tb1 = tb[tb['Trade_Type'] == 'Long']
tb2 = tb[tb['Trade_Type'] == 'Add_Long']
tb1['ROI%'].sum() , tb2['ROI%'].sum()

(521.269890144499, 0)

In [46]:
tb.tail(25)

,index,Trade No.,Entry_Date,Exit_Date,Trade_Type,Entry_Price,Previous MA Value,Initial SL,Exit_Price,Exit Remark,PnL,Slippage,PnL w cs,Qty,Profit_Loss INR,ROI%,Trade Year
406,406,406,2024-06-03 09:15:00+05:30,2024-06-04 09:15:00+05:30,Long,23197.1000,22606.6833,22680.4167,22680.4167,Initial SL Hit,-516.6833,9.1755,-525.8588,580.6264,-305327.5398,-3.0533,2024
407,407,407,2024-06-05 13:15:00+05:30,2024-06-20 09:15:00+05:30,Long,22475.7000,22406.9833,22269.4667,23487.3500,Trailing SL Hit,1011.6500,9.1926,1002.4574,1454.6630,1458237.6774,14.5824,2024
408,408,408,2024-06-25 12:15:00+05:30,2024-07-05 09:15:00+05:30,Long,23672.5000,23609.3500,23574.1500,24226.0500,Trailing SL Hit,553.5500,9.5797,543.9703,3050.3305,1659289.1408,16.5929,2024
409,409,409,2024-07-05 14:15:00+05:30,2024-07-08 11:15:00+05:30,Long,24342.0500,24281.0333,24227.7333,24252.6500,Trailing SL Hit,-89.4000,9.7189,-99.1189,2624.2893,-260116.7692,-2.6012,2024
410,410,410,2024-07-12 10:15:00+05:30,2024-07-18 10:15:00+05:30,Long,24538.2500,24376.7333,24335.6167,24537.3500,Trailing SL Hit,-0.9000,9.8151,-10.7151,1480.5067,-15863.8065,-0.1586,2024
411,411,411,2024-07-18 12:15:00+05:30,2024-07-19 12:15:00+05:30,Long,24711.6000,24649.7000,24545.2333,24612.8000,Trailing SL Hit,-98.8000,9.8649,-108.6649,1803.2458,-195949.4931,-1.9595,2024
412,412,412,2024-07-26 09:15:00+05:30,2024-08-02 09:15:00+05:30,Long,24578.7000,24424.6500,24393.3167,24778.9000,Trailing SL Hit,200.2000,9.8715,190.3285,1618.2685,308002.5748,3.0800,2024
413,413,413,2024-08-09 09:15:00+05:30,2024-08-12 09:15:00+05:30,Long,24356.0000,24164.4167,24169.6500,24252.1500,Trailing SL Hit,-103.8500,9.7216,-113.5716,1609.8739,-182836.0021,-1.8284,2024
414,414,414,2024-08-12 10:15:00+05:30,2024-08-13 09:15:00+05:30,Long,24434.8000,24362.4500,24273.7333,24330.1000,Trailing SL Hit,-104.7000,9.7530,-114.4530,1862.5828,-213178.1498,-2.1318,2024
415,415,415,2024-08-16 09:15:00+05:30,2024-09-04 09:15:00+05:30,Long,24295.0000,24171.6333,24175.3000,25152.4500,Trailing SL Hit,857.4500,9.8895,847.5605,2506.2657,2124211.8045,21.2421,2024


In [47]:
# tb.to_csv('SENSEX_jjpp_55_9_HALF_PCT_0_PYR.csv')

In [48]:
# tb_long_only.to_csv('GOLD Range JJPP.csv', index=False)

In [50]:
# ema_length = 55
# x_days = 28
# y_days = 7
# pct = 1.05
# RPT = 5
RPT_ = 3
pct_range = [0.25, 0.5, 0.75, 1]
stats_dictionary = {}

for ema_length in range(20, 81, 2):
    for x_days in range(3, 37, 3):
            for pct in pct_range:
                # RPT = 5
                variation = f'EMA:{ema_length}, X:{x_days}, PCT:{pct}'
                print(variation)
                df = bnf_1min
                df = calculate_signals(df, ema_length=ema_length, x_days=x_days, pct=pct)
                tb = backtest(df)
                if len(tb) > 0:
                    tb = tb.sort_values(by='Entry_Date')
                    portfolio = 10000000
                    tb['PnL'] = np.where(
                        tb['Trade_Type'].isin(['Long', 'Add_Long']),  # For Long and Add_Long
                        tb['Exit_Price'] - tb['Entry_Price'],
                        np.where(
                            tb['Trade_Type'].isin(['Short', 'Add_Short']),  # For Short and Add_Short
                            tb['Entry_Price'] - tb['Exit_Price'],
                            0  # Default case if Trade_Type is something unexpected
                        )
                    )
                    
                    tb['Slippage'] = SLIPPAGE_ * (tb['Entry_Price'] + tb['Exit_Price'])
                    tb['PnL w cs'] = tb['PnL'] - tb['Slippage']
                    tb['Qty'] = abs(RPT_ / 100 * portfolio / (tb['Entry_Price'] - tb['Initial SL']))
                    tb['Profit_Loss INR'] = tb['Qty'] * tb['PnL w cs']
                    tb['ROI%'] = tb['Profit_Loss INR'] * 100 / portfolio
                    tb['Entry_Date'] = pd.to_datetime(tb['Entry_Date'])
                    tb['Trade Year'] = tb['Entry_Date'].dt.year
                    # tb_long_only = tb[(tb['Trade_Type'] == 'Long') | (tb['Trade_Type'] == 'Add_Long')].copy()
                    # tb_long_only['Cumulative_PnL'] = tb_long_only['PnL w cs'].cumsum()
                    # tb_short_only = tb[(tb['Trade_Type'] == 'Short') | (tb['Trade_Type'] == 'Add_Short')].copy()
                    # tb_short_only['Cumulative_PnL'] = tb_short_only['PnL w cs'].cumsum()
                    
                    stats1 = generate_stats(tb, variation)
                    for x, y in stats1.items():
                        if x > 20:
                            # print('Long Only')
                            print(pd.DataFrame(y).to_string())
                            stats_dictionary[x] = y
                        break

EMA:20, X:3, PCT:0.25


/tmp/ipykernel_32292/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio              Variation
2017      41.9279           57  35.0877                5.9858             -2.1024     -14.2530       2.9417  EMA:20, X:3, PCT:0.25
2018      58.9088           46  45.6522                5.3542             -2.1412     -16.0640       3.6671  EMA:20, X:3, PCT:0.25
2019      42.9348           55  32.7273                6.4110             -1.9584     -25.5116       1.6830  EMA:20, X:3, PCT:0.25
2020     111.5422           60  40.0000                8.0413             -2.2624     -14.3503       7.7728  EMA:20, X:3, PCT:0.25
2021      71.0830           62  37.0968                6.1513             -1.8051     -24.9277       2.8516  EMA:20, X:3, PCT:0.25
2022      44.1738           55  30.9091                7.1224             -2.0239     -23.8881       1.8492  EMA:20, X:3, PCT:0.25
2023     100.8319           48  43.7500                7.5237             -2.1172  

/tmp/ipykernel_32292/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:20, X:3, PCT:0.75


...

/tmp/ipykernel_32292/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:0.75


/tmp/ipykernel_32292/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


EMA:80, X:36, PCT:1


/tmp/ipykernel_32292/2639050160.py:92: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'Trailing SL Hit' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  tradebook_long.loc[


# NIFTY 1Hr

In [52]:
sorted_stats = {k: v for k, v in sorted(stats_dictionary.items(), key=lambda item: item[0], reverse=True)}
for x, y in sorted_stats.items():
    print(y.to_string())

        Total ROI Total Trades Win Rate Avg Profit% per Trade Avg Loss% per Trade Max Drawdown ROI/DD Ratio               Variation
2017      54.7871           31  48.3871                5.7417             -1.9586     -10.4328       5.2514  EMA:46, X:18, PCT:0.25
2018      74.8003           31  41.9355                7.5247             -1.2789      -5.9897      12.4882  EMA:46, X:18, PCT:0.25
2019      23.5856           38  34.2105                5.6507             -1.9949     -12.5198       1.8839  EMA:46, X:18, PCT:0.25
2020      81.4779           34  47.0588                7.3119             -1.9729      -8.4793       9.6090  EMA:46, X:18, PCT:0.25
2021      66.2804           38  31.5789                8.7436             -1.4863     -14.6697       4.5182  EMA:46, X:18, PCT:0.25
2022      36.3959           36  27.7778                7.7252             -1.5714     -12.3253       2.9529  EMA:46, X:18, PCT:0.25
2023      69.6313           34  35.2941                8.8632             -1